# Multi-Agent Belief Sharing Problem: Notebook Overview

Solve a simple multi-agent information-sharing planning problem using Spectra (SST_Prover + run_spectra).

## Domain & Background
- Domain objects: `a`, `b`, `c` (agents); `p1`, `p2` (rooms).
- Background facts and axioms include:
    - Agent/room declarations: `(agent a)`, `(agent b)`, `(agent c)`, `(room p1)`, `(room p2)`.
    - Inequality/uniqueness: `(not (= a b))`, `(not (= a c))`, `(not (= b c))`.

## Actions
Four parameterized actions are defined:
- `left ?a` — move agent `?a` from `p2` to `p1` (precondition: at `p2`); effects update `(at ?a p1)` / `(not (at ?a p2))`.
- `right ?a` — move agent `?a` from `p1` to `p2` (precondition: at `p1`); effects update `(at ?a p2)` / `(not (at ?a p1))`.
- `shareboth ?a1 ?a2 ?a3 ?r` — when three distinct agents meet in room `?r`, `?a1` causes `?a2` and `?a3` to believe `the ?a1`, and `?a1` to believe that `?a2`/`?a3` believe `the ?a1`. Adds corresponding `Believes!` facts and removes their negations.
- `sharesingle ?a1 ?a2 ?a3 ?r` — when `?a1` and `?a2` meet in `?r` (and `?a3` is not there), `?a1` causes `?a2` to believe `the ?a1` and `?a1` to believe that `?a2` believes `the ?a1`. Adds corresponding `Believes!` facts and removes their negations.

Each action encodes preconditions, addition effects, and deletion effects (including toggling negated belief facts).

## Initial State and Goal
- Initial state (`start`) includes agents at `p1`:
    - `(at a p1)`, `(at b p1)`, `(at c p1)` (and corresponding `(not (at X p2))`), and initial belief facts like `(Believes! a (the a))`, `(Believes! b (the b))`, `(Believes! c (the c))` plus negated cross-beliefs.
- Goal (`goal`):
    - `(and (Believes! b (the a)) (Believes! a (Believes! b (the a))) (not (Believes! c (the a))))`

## Plan Found
A plan of length 2 was found (sequence executed in order):
1. `(right c)` — move agent `c` to `p2`.
2. `(sharesingle a b c p1)` — `a` shares with `b` in `p1` while `c` is absent.

Solver details: plan produced by `run_spectra(...)` using `SST_Prover`'s cached shadow prover.

In [1]:
import os

os.environ["EPROVER_HOME"] = "./eprover/"
from shadowprover.syntax import *
from shadowprover.reasoners.planner import Action
from shadowprover.syntax.reader import r


from shadowprover.experimental.sst_prover import SST_Prover
from shadowprover.reasoners.planner import run_spectra


/home/eichc/miniconda3/envs/spectra_env/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Domain Definition

In [2]:
domain = {r("a"), r("b"), r("c")}

background = set(
    list(
        map(
            r,
            [
                "(agent a)",
                "(agent b)",
                "(agent c)",
                "(room p1)",
                "(room p2)",
                "(not (= a b))",
                "(not (= a c))",
                "(not (= b c))",
            ],
        )
    )
)

actions = [
    Action(
        r("(left ?a)"),
        precondition=r("(and (agent ?a) (at ?a p2))"),
        additions={r("(at ?a p1)"), r("(not (at ?a p2))")},
        deletions={r("(at ?a p2)"), r("(not (at ?a p1))")},
    ),
    Action(
        r("(right ?a)"),
        precondition=r("(and (agent ?a) (at ?a p1))"),
        additions={r("(at ?a p2)"), r("(not (at ?a p1))")},
        deletions={r("(at  ?a p1)"), r("(not (at ?a p2))")},
    ),
    Action(
        r("(shareboth ?a1 ?a2 ?a3 ?r)"),
        precondition=r(
            """(and 
                    (agent ?a1)
                    (agent ?a2)
                    (agent ?a3)
                    (room ?r)
                    ; Precondition
                    (at ?a1 ?r)
                    (at ?a2 ?r)
                    (at ?a3 ?r)
                    (not (= ?a1 ?a2))
                    (not (= ?a1 ?a3))
                    (not (= ?a2 ?a3))
            )"""
        ),
        additions={
            r("(Believes! ?a2 (the ?a1))"),
            r("(Believes! ?a3 (the ?a1))"),
            r("(Believes! ?a1 (Believes! ?a2 (the ?a1)))"),
            r("(Believes! ?a1 (Believes! ?a3 (the ?a1)))"),
        },
        deletions={
            r("(not (Believes! ?a2 (the ?a1)))"),
            r("(not (Believes! ?a3 (the ?a1)))"),
            r("(not (Believes! ?a1 (Believes! ?a2 (the ?a1))))"),
            r("(not (Believes! ?a1 (Believes! ?a3 (the ?a1))))"),
        },
    ),
    Action(
        r("(sharesingle ?a1 ?a2 ?a3 ?r)"),
        precondition=r(
            """(and 
                    (agent ?a1)
                    (agent ?a2)
                    (agent ?a3)
                    (room ?r)
                    ; Precondition
                    (at ?a1 ?r)
                    (at ?a2 ?r)
                    (not (at ?a3 ?r))
                    (not (= ?a1 ?a2))
                    (not (= ?a1 ?a3))
                    (not (= ?a2 ?a3))
            )"""
        ),
        additions={
            r("(Believes! ?a2 (the ?a1))"),
            r("(Believes! ?a1 (Believes! ?a2 (the ?a1)))"),
        },
        deletions={
            r("(not (Believes! ?a2 (the ?a1)))"),
            r("(not (Believes! ?a1 (Believes! ?a2 (the ?a1))))"),
        },
    ),
]

start = set(
    map(
        r,
        [
            "(at a p1)",
            "(not (at a p2))",
            "(at b p1)",
            "(not (at b p2))",
            "(at c p1)",
            "(not (at c p2))",
            "(Believes! a (the a))",
            "(Believes! b (the b))",
            "(Believes! c (the c))",
            "(not (Believes! a (the b)))",
            "(not (Believes! a (the c)))",
            "(not (Believes! b (the a)))",
            "(not (Believes! b (the c)))",
            "(not (Believes! c (the a)))",
            "(not (Believes! c (the b)))",
        ],
    )
)

goal = r(
    """(and 
                (Believes! b (the a)) 
                (Believes! a (Believes! b (the a)))
                 (not (Believes! c (the a)))
                )"""
)

### Initialize Reasoner

In [3]:
sst = SST_Prover()

### Generate Plan

In [4]:
goal = r(
    """(and 
                (Believes! b (the a)) 
                (Believes! a (Believes! b (the a)))
                 (not (Believes! c (the a)))
                )"""
)
result = run_spectra(
    domain,
    background,
    start,
    goal,
    actions,
    sst.get_cached_shadow_prover2(),
    verbose=False,
)[0]

In [5]:
result

[(right c), (sharesingle a b c p1)]